# TESTING HYPERPARAMETERS
# TRANSFORMER. 8 featerow - mag, x, y, depth, dist, distance, plate, days_diff. 
### num_layers=2, d_model=48, num_heads=4

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
SEED = 1337
tf.random.set_seed(SEED)
np.random.seed(SEED)

2024-01-01 13:23:07.814658: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-01 13:23:07.814783: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-01 13:23:07.817976: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-01 13:23:08.059468: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-01 13:23:10.123721: W tensorflow/compiler/

In [2]:
ds_train = tf.data.Dataset.load("../data/ds_train/")
ds_val = tf.data.Dataset.load("../data/ds_val/")
ds_test = tf.data.Dataset.load("../data/ds_test/")

2024-01-01 13:23:14.778748: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-01 13:23:15.136879: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-01 13:23:15.136962: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-01 13:23:15.143284: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-01 13:23:15.143396: I tensorflow/compile

In [3]:
ds_train.cardinality().numpy(), ds_val.cardinality().numpy(), ds_test.cardinality().numpy()

(3294404, 465661, 101326)

In [4]:
BATCH_SIZE = 1024
BLOCK_SIZE = 64

In [5]:
def fix_x_and_y(features, context, labels):
    return features, labels

In [6]:
ds_train = ds_train.map(fix_x_and_y)
ds_val = ds_val.map(fix_x_and_y)
ds_test = ds_test.map(fix_x_and_y)

In [7]:
for x, y in ds_train.batch(BATCH_SIZE).take(1):
    print(x[0].shape)
    print(x[1].shape)
    print(y.shape)

(64, 9)
(64, 9)
(1024, 1)


In [8]:
# ys = []
# for x, y in ds_train.batch(BATCH_SIZE):
#     ys.append(y.numpy())
# ys = np.concatenate(ys)
# np.bincount(tf.cast(ys, tf.int32).numpy().flatten())

In [9]:
#neg, pos = np.bincount(tf.cast(ys, tf.int32).numpy().flatten())
neg, pos = 3094535, 199869
total = neg + pos
total, pos, neg

(3294404, 199869, 3094535)

In [10]:
ds_train = ds_train.batch(BATCH_SIZE).shuffle(2000, seed=SEED, reshuffle_each_iteration=False).prefetch(tf.data.experimental.AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [11]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, d_model):
        super().__init__()
        self.dense = tf.keras.layers.Dense(d_model // 9 * 6)
        self.embed_dd = tf.keras.layers.Embedding(20, d_model // 9)
        self.embed_plate = tf.keras.layers.Embedding(64, d_model // 9)
        self.embed_magtype = tf.keras.layers.Embedding(20, d_model // 9)
        self.conc = tf.keras.layers.Concatenate()
        self.pos_encoding = tf.keras.layers.Embedding(64, d_model)

    def call(self, x):
        cont, plate, dd, magtype = x[:,:,:-3], x[:,:,-3], x[:,:,-2], x[:,:,-1] 
        x1 = self.dense(cont)
        x2 = self.embed_dd(dd)
        x3 = self.embed_plate(plate)
        x4 = self.embed_magtype(magtype)
        x = self.conc([x1, x2, x3, x4])
        x_pos = self.pos_encoding(tf.range(x1.shape[1]))
        x = x + x_pos
        return x

In [12]:
class BaseAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()

In [13]:
class GlobalSelfAttention(BaseAttention):
    def call(self, x):
        attn_output = self.mha(
            query=x,
            value=x,
            key=x)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x

In [14]:
class FeedForward(tf.keras.layers.Layer):
    def __init__(self, d_model, dff, dropout_rate=0.1):
        super().__init__()
        self.seq = tf.keras.Sequential([
            tf.keras.layers.Dense(dff, activation='relu'),
            tf.keras.layers.Dense(d_model),
            tf.keras.layers.Dropout(dropout_rate)
        ])
        self.add = tf.keras.layers.Add()
        self.layer_norm = tf.keras.layers.LayerNormalization()

    def call(self, x):
        x = self.add([x, self.seq(x)])
        x = self.layer_norm(x) 
        return x

In [15]:
class DecoderBlock(tf.keras.layers.Layer):
    def __init__(self,
                d_model,
                num_heads,
                dff,
                dropout_rate=0.1):
        super(DecoderBlock, self).__init__()

        self.causal_self_attention = GlobalSelfAttention(
            num_heads=num_heads,
            key_dim=d_model,
            dropout=dropout_rate)

        self.ffn = FeedForward(d_model, dff)

    def call(self, x):
        x = self.causal_self_attention(x=x)
        x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
        return x

In [16]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, dropout_rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.pos_embedding = PositionalEmbedding(d_model=d_model)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.dec_blocks = [
            DecoderBlock(d_model=d_model, num_heads=num_heads,
                        dff=dff, dropout_rate=dropout_rate)
            for _ in range(num_layers)]

    def call(self, x):
        x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)
        x = self.dropout(x)
        for block in self.dec_blocks:
            x = block(x)
        # The shape of x is (batch_size, target_seq_len, d_model).
        return x

In [20]:
@tf.keras.utils.register_keras_serializable()
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads,
                dropout_rate=0.1):
        super().__init__()
        self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                            num_heads=num_heads, dff=4*d_model,
                            dropout_rate=dropout_rate)

        self.final_layer = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.decoder(inputs) # (batch_size, target_len, d_model)
        x = x[:, -1, :]
        logits = self.final_layer(x)  # (batch_size, 1, target_vocab_size)
        return logits

In [27]:
model = Transformer(num_layers=2, d_model=99, num_heads=2)

In [28]:
model.build(input_shape=(None, BLOCK_SIZE, 9))
model.count_params()

324842

In [29]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
            optimizer='adam',
            metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

In [30]:
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}
history = model.fit(ds_train, epochs=20, validation_data=ds_val)

Epoch 1/20


2024-01-01 13:24:16.832041: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 40 of 2000
2024-01-01 13:24:36.784653: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 111 of 2000
2024-01-01 13:24:46.904062: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 156 of 2000
2024-01-01 13:25:06.893338: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 248 of 2000
2024-01-01 13:25:26.822146: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 372 of 2000
2024-01-01 13:26:06.776828: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a w

3218/3218 [==============================] - 1351s 236ms/step - loss: 0.4777 - accuracy: 0.8189 - recall: 0.7485 - precision: 0.2149 - val_loss: 0.4560 - val_accuracy: 0.8585 - val_recall: 0.5287 - val_precision: 0.2270
Epoch 2/20


2024-01-01 13:46:42.135597: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 82 of 2000
2024-01-01 13:46:52.183053: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 161 of 2000
2024-01-01 13:47:12.140694: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 380 of 2000
2024-01-01 13:47:22.163846: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 485 of 2000
2024-01-01 13:47:32.189270: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 574 of 2000
2024-01-01 13:47:42.200969: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a w

3218/3218 [==============================] - 1114s 281ms/step - loss: 0.4371 - accuracy: 0.8471 - recall: 0.7586 - precision: 0.2497 - val_loss: 0.4524 - val_accuracy: 0.8730 - val_recall: 0.5013 - val_precision: 0.2456
Epoch 3/20


...

3218/3218 [==============================] - 1347s 329ms/step - loss: 0.3348 - accuracy: 0.8585 - recall: 0.8480 - precision: 0.2801 - val_loss: 0.4833 - val_accuracy: 0.8016 - val_recall: 0.8038 - val_precision: 0.2119
Epoch 19/20


2024-01-01 19:26:11.274151: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 91 of 2000
2024-01-01 19:26:31.286127: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 247 of 2000
2024-01-01 19:26:41.296746: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 316 of 2000
2024-01-01 19:26:51.330651: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 380 of 2000
2024-01-01 19:27:11.282268: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 515 of 2000
2024-01-01 19:27:21.348591: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a w

3218/3218 [==============================] - 1096s 253ms/step - loss: 0.3345 - accuracy: 0.8580 - recall: 0.8496 - precision: 0.2796 - val_loss: 0.4518 - val_accuracy: 0.8202 - val_recall: 0.7879 - val_precision: 0.2275
Epoch 20/20


2024-01-01 19:44:27.750896: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 70 of 2000
2024-01-01 19:44:37.786254: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 139 of 2000
2024-01-01 19:44:57.727411: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 266 of 2000
2024-01-01 19:45:07.734406: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 331 of 2000
2024-01-01 19:45:17.782376: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 393 of 2000
2024-01-01 19:45:27.798543: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a w

3218/3218 [==============================] - 1072s 244ms/step - loss: 0.3283 - accuracy: 0.8593 - recall: 0.8534 - precision: 0.2821 - val_loss: 0.4415 - val_accuracy: 0.8258 - val_recall: 0.7202 - val_precision: 0.2224


In [31]:
model.evaluate(ds_test)

99/99 [==============================] - 14s 136ms/step - loss: 0.4411 - accuracy: 0.8248 - recall: 0.7938 - precision: 0.0950


[0.44113630056381226,
 0.8248327374458313,
 0.7937828302383423,
 0.09496621787548065]

In [32]:
model.save("../models/model_v4.keras")

In [33]:
# save history
history = pd.DataFrame.from_dict(history.history)
history.to_csv("../models/history_model_v4.csv", index=False)